<a href="https://colab.research.google.com/github/decipher6/WorkoutWithWeather/blob/main/WorkoutWithWeather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import requests
from google.colab import userdata # Secret API keys

CITY = 'New York' #The user may change this parameter

In [11]:
def get_weather_data(api_key, city):

    # gets weather data from the OpenWeatherMap API

    # URL for the OpenWeatherMap API
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    # parameters for the API request
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    # request to the OpenWeatherMap API with the specified parameters
    response = requests.get(base_url, params=params)

    # returns the JSON response from the API
    return response.json()

In [24]:
def get_exercise_suggestions():

    # gets exercise list from ExerciseDB API

    XRapidAPIKey = userdata.get('X-RapidAPI-Key')
    url = "https://exercisedb.p.rapidapi.com/exercises"
    headers = {
        "X-RapidAPI-Host": "exercisedb.p.rapidapi.com",
        "X-RapidAPI-Key": XRapidAPIKey
    }

    exercises = []
    page = 0
    more_data = True

    while more_data:
        params = {
            'offset': page * 50,  # Assuming 50 exercises per page
            'limit': 50           # Fetch 50 exercises per request
        }
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()  # Check for HTTP errors
            data = response.json()

            if data:
                exercises.extend(data)  # Add fetched exercises to the list
                page += 1  # Move to the next page
            else:
                more_data = False  # No more data to fetch
        except requests.exceptions.RequestException as e:
            print(f"Error fetching exercises: {e}")
            more_data = False

    return exercises

In [29]:
def get_fitness_recommendations(weather_data, exercises):

    # extract temperature and weather description from the weather data
    temp = weather_data['main']['temp']
    weather_description = weather_data['weather'][0]['description']

    # list of indoor equipments
    indoor_equipment = {
        'assisted', 'band', 'barbell', 'bosu ball', 'cable', 'dumbbell', 'elliptical machine', 'ez barbell', 'hammer', 'kettlebell',
        'leverage machine', 'medicine ball', 'olympic barbell', 'roller', 'skierg machine', 'sled machine', 'smith machine',
        'stability ball', 'stationary bike', 'stepmill machine', 'trap bar', 'upper body ergometer', 'weighted', 'wheel roller'
    }

    # list of outdoor equipments
    outdoor_equipment = {'body weight', 'rope', 'resistance band', 'tire'}

    # switch everything to lowercase for comparison
    indoor_equipment = {eq.lower() for eq in indoor_equipment}
    outdoor_equipment = {eq.lower() for eq in outdoor_equipment}

    indoor_exercises = []
    outdoor_exercises = []

    # use exercise equipment to categorize indoor and outdoor exercises
    for exercise in exercises:
        eq = exercise['equipment']

        if (eq in indoor_equipment):
            indoor_exercises.append(exercise['name'])
        if (eq in outdoor_equipment):
            outdoor_exercises.append(exercise['name'])

    # recommend based on temperature and weather description
    if temp > 30:
        recommendation = "It's hot outside. Consider indoor exercises."
        suggestions = indoor_exercises
    elif temp < 0:
        recommendation = "It's freezing outside. Consider indoor exercises."
        suggestions = indoor_exercises
    elif weather_description in ['rain', 'light rain', 'drizzle', 'thunderstorm']:
        recommendation = "It's rainy outside. Indoor workouts might be a better choice."
        suggestions = indoor_exercises
    elif weather_description in ['clear sky', 'few clouds', 'scattered clouds']:
        recommendation = "The weather is clear. Great day for an outdoor workout!"
        suggestions = outdoor_exercises
    else:
        recommendation = "Check the weather conditions. Adjust your workout plan accordingly."
        suggestions = indoor_exercises + outdoor_exercises

    return recommendation, suggestions

In [30]:
def main():

    # retrieve the Weather API key from secrets
    WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')

    # fetch weather data
    weather_data = get_weather_data(WEATHER_API_KEY, CITY)

    # if the API response is successful
    if weather_data['cod'] == 200:

        # get exercose recommendations and suggestions
        exercises = get_exercise_suggestions()
        recommendation, suggestions = get_fitness_recommendations(weather_data, exercises)

        # print the weather conditions and the exercise suggestions
        print(f"Weather in {CITY}:")
        print(f"Temperature: {weather_data['main']['temp']}°C")
        print(f"Weather: {weather_data['weather'][0]['description']}")
        print(f"Humidity: {weather_data['main']['humidity']}%")
        print(f"Fitness Recommendation: {recommendation}")
        print(f"Suggested Exercises:")
        for exercise in suggestions:
            print(f"- {exercise}")
    else:
        print("Error fetching weather data.")

if __name__ == "__main__":
    main()


Weather in New York:
Temperature: 27.56°C
Weather: few clouds
Humidity: 40%
Fitness Recommendation: The weather is clear. Great day for an outdoor workout!
Suggested Exercises:
- 3/4 sit-up
- 45° side bend
- air bike
- all fours squad stretch
- alternate heel touchers
- ankle circles
- archer pull up
- archer push up
- arm slingers hanging bent knee legs
- arm slingers hanging straight legs
- arms apart circular toe touch (male)
- arms overhead full sit-up (male)
- astride jumps (male)
- back and forth step
- back lever
- back pec stretch
- backward jump
- balance board
- basic toe touch (male)
- battling ropes
- bear crawl
- bench dip (knees bent)
- bench dip on floor
- bench hip extension
- bench pull-ups
- bent knee lying twist (male)
- biceps leg concentration curl
- biceps narrow pull-ups
- biceps pull-up
- body-up
- bodyweight drop jump squat
- bodyweight incline side plank
- bodyweight kneeling triceps extension
- bodyweight side lying biceps curl
- bodyweight squatting row
- bo